# 2. データパイプライン


<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">

このデータパイプラインについて、ソクラテス式チャットボットの文脈で説明していきます。

### 1. トークナイザー設定 (2.1)
```python
model_name = "google/gemma-2-2b-jpn-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)
```
これは、文章を機械が理解できる数値（トークン）に変換するツールを設定しています。

例えば：
- ユーザー: 「なぜ空は青いのですか？」
- ボット: 「その質問は興味深いですね。一緒に考えてみましょう。」

という会話を、機械が理解できる数値の列に変換します。また、「。」「？」などの句読点も特別なトークンとして認識するように設定しています。

### 2. データセットの準備 (2.2)
```python
def prepare_dataset():
    conversations = []
    # ...
```
この部分は、学習用の会話データを整理する工程です。以下のようなチェックを行います：

1. **フォーマットの確認**：
   - 各メッセージが正しい形式（ユーザーとボットの役割、内容があるか）かチェック

2. **会話の順序確認**：
   ```python
   messages[i]['role'] == 'user' and messages[i+1]['role'] == 'model'
   ```
   - ユーザーの質問→ソクラテス式の返答→ユーザーの質問→...
   という正しい順序になっているか確認

3. **長さの確認**：
   - 会話が長すぎないかチェック（256トークンまで）

例えば：
```json
{
    "messages": [
        {"role": "user", "content": "なぜ勉強は大切なのでしょうか？"},
        {"role": "model", "content": "その質問について、あなたはどう考えていますか？まずはあなたの考えを聞かせてください。"}
    ]
}
```

### 3. トークン化処理 (2.3, 2.4)
```python
def tokenize_function(examples):
    result = tokenizer(examples['text'],...)
```
準備された会話データを実際に数値に変換します：

1. **トークン化**：
   - 文章→数値列に変換
   - 例：「なぜ」→[245, 876]のような数値の列に

2. **長さの調整**：
   - すべての会話を同じ長さ（256）に調整
   - 短い会話は特殊なパディングトークンで埋める
   - 長い会話は切り詰める

3. **検証**：
   - 変換されたデータが正しい形式になっているか確認

このように処理されたデータは、モデルが学習できる形式になり、ソクラテス式の対話方法を学習する準備が整います。

        
</div>
    </details>
</div>


In [ ]:
### 2.1 トークナイザー設定
# Model and tokenizer preparation
model_name = "google/gemma-2-2b-jpn-it"
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    token=os.environ["HUGGINGFACE_TOKEN"],  
    trust_remote_code=True
)


# Add special tokens to tokenizer
tokenizer.add_special_tokens({
    'additional_special_tokens': [
        '。', '、', '！', '？',  # Punctuation marks
    ]
})


<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">

# トークナイザー設定の説明

トークナイザー（Tokenizer）について、ソクラテス式チャットボットの具体例を交えて説明させていただきます。

## 1. トークナイザーとは？
トークナイザーは、テキストを小さな単位（トークン）に分割するツールです。人間が読む文章をAIが理解できる形に変換する「通訳」のような役割を果たします。

### 具体例：
以下のような会話があったとします：
```
ユーザー：「なぜ人は学ぶのでしょうか？」
ソクラテスボット：「それは興味深い質問ですね。あなたはどう考えますか？」
```

トークナイザーは、この文章を以下のように分割します：
- 「なぜ」「人」「は」「学ぶ」「の」「でしょう」「か」「？」

## 2. コードの説明

```python
model_name = "google/gemma-2-2b-jpn-it"
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    token=os.environ["HUGGINGFACE_TOKEN"],  
    trust_remote_code=True
)
```

このコードでは：
1. `google/gemma-2-2b-jpn-it`という日本語対応のAIモデルを使用
2. `AutoTokenizer.from_pretrained()`で、このモデル用の専用トークナイザーを読み込み
3. `HUGGINGFACE_TOKEN`で認証を行い、モデルにアクセス

## 3. 特殊トークンの追加

```python
tokenizer.add_special_tokens({
    'additional_special_tokens': [
        '。', '、', '！', '？',  # Punctuation marks
    ]
})
```

このコードでは：
- 日本語の句読点（「。」「、」）や感嘆符（「！」「？」）を特別なトークンとして追加
- これにより、ソクラテスボットの返答がより自然な日本語の区切りを持つように

### 具体例：
```
「哲学とは何だと思いますか？」
↓ トークナイザーが処理
「哲学」「と」「は」「何」「だ」「と」「思い」「ます」「か」「？」
```

このように、文章を適切に区切ることで、AIがより正確に文章の意味を理解し、ソクラテス式の対話を自然に行うことができるようになります。

        
</div>
    </details>
</div>


In [ ]:

### 2.2 データセット準備と検証
#""""""""""""""""""""""""""
def validate_message_format(message):
    """メッセージのフォーマットを検証"""
    if not isinstance(message, dict):
        return False
    if 'role' not in message or 'content' not in message:
        return False
    if message['role'] not in ['user', 'model']:
        return False
    if not isinstance(message['content'], str):
        return False
    return True

def prepare_dataset():
    conversations = []
    
    try:
        with open(DIALOGUE_JSON_PATH, 'r', encoding='utf-8') as f:
            dialogue_data = json.load(f)
            
        for dialogue in dialogue_data:
            messages = dialogue.get('messages', [])
            
            # メッセージのフォーマットを検証
            if not all(validate_message_format(msg) for msg in messages):
                logging.warning(f"Skipped dialogue due to invalid message format")
                continue
                
            # user->modelの順序を確認しながら会話を構築
            current_conversation = []
            valid_sequence = True
            
            for i in range(0, len(messages)-1, 2):
                if (i+1 < len(messages) and 
                    messages[i]['role'] == 'user' and 
                    messages[i+1]['role'] == 'model'):
                    current_conversation.extend([messages[i], messages[i+1]])
                else:
                    valid_sequence = False
                    break
            
            # 有効な会話のみを追加
            if valid_sequence and current_conversation:
                # Gemmaのチャットテンプレートを適用
                formatted_text = tokenizer.apply_chat_template(
                    current_conversation,
                    tokenize=False,
                    add_generation_prompt=True
                )
                
                # トークン数をチェック
                tokens = tokenizer.encode(formatted_text)
                if len(tokens) <= MAX_SEQUENCE_LENGTH:
                    conversations.append({"text": formatted_text})
                else:
                    logging.warning(f"Skipped conversation due to length: {len(tokens)} tokens")
            
    except Exception as e:
        logging.error(f"Error processing dialogue file: {str(e)}")
        raise
    
    if not conversations:
        raise ValueError("No valid conversations found in the dialogue file")
        
    logging.info(f"Processed {len(conversations)} valid conversations")
    return Dataset.from_list(conversations)
dataset = prepare_dataset()
#""""""""""""""""""""""""""
# データセットの構造を確認
print("Dataset structure:")
print(dataset[0])  # 最初の要素を表示
print("\nDataset features:")
print(dataset.features)

# データセットのバッチ処理を最適化
dataset = dataset.select(range(len(dataset))).shuffle(seed=42)


<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">



# データセット準備と検証の説明

このコードは、ソクラテス式チャットボットの学習データを準備し、その品質を確保するための重要な部分です。

## 1. メッセージフォーマットの検証関数

```python
def validate_message_format(message):
    """メッセージのフォーマットを検証"""
    if not isinstance(message, dict):
        return False
    if 'role' not in message or 'content' not in message:
        return False
    if message['role'] not in ['user', 'model']:
        return False
    if not isinstance(message['content'], str):
        return False
    return True
```

この関数は、各メッセージが正しい形式かチェックします：

### 具体例：
✅ 正しい形式：
```python
{
    "role": "user",
    "content": "知識とは何だと思いますか？"
}
```

❌ 間違った形式：
```python
{
    "speaker": "user",  # "role"ではないので不正
    "text": "知識とは何だと思いますか？"  # "content"ではないので不正
}
```

## 2. データセット準備関数

```python
def prepare_dataset():
    conversations = []
    # ... 既存のコード ...
```

この関数は以下の重要な処理を行います：

### a) 会話データの読み込みと検証
- JSONファイルから会話データを読み込み
- 各メッセージの形式を検証
- ユーザーとモデルの発言が正しく交互になっているか確認

### b) 会話形式の整形
```python
formatted_text = tokenizer.apply_chat_template(
    current_conversation,
    tokenize=False,
    add_generation_prompt=True
)
```

### 具体例：
変換前：
```python
[
    {"role": "user", "content": "正義とは何ですか？"},
    {"role": "model", "content": "その質問について、あなたはどう考えますか？"}
]
```

変換後：
```
<start>ユーザー: 正義とは何ですか？
システム: その質問について、あなたはどう考えますか？<end>
```

### c) 長さの確認
- 会話が長すぎないかチェック（`MAX_SEQUENCE_LENGTH`以下）
- 長すぎる会話は学習データから除外

## 3. データセットの最終処理

```python
dataset = dataset.select(range(len(dataset))).shuffle(seed=42)
```

- データをランダムに並び替え（シャッフル）
- 42というシード値を使用して、毎回同じ順序になるように制御

このような厳密な検証と準備により、高品質な学習データセットが作成され、ソクラテス式チャットボットの応答の質を向上させることができます。

        
</div>
    </details>
</div>


In [ ]:

### 2.3 トークン化関数の定義

def tokenize_function(examples):
    result = tokenizer(
        examples['text'],
        truncation=True,
        max_length=TOKENIZE_MAX_LENGTH,      # 256 から TOKENIZE_MAX_LENGTH に変更
        padding='max_length',
        add_special_tokens=True,
        return_tensors=None,
    )
    return result

# Add dataset preprocessing
def preprocess_function(examples):
    """シンプルな前処理"""
    return tokenizer(
        examples['text'],
        truncation=True,
        max_length=TOKENIZE_MAX_LENGTH,
        padding='max_length',
        add_special_tokens=True,
        return_tensors=None
    )




<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">



# トークン化関数の説明

このコードでは、テキストデータをAIモデルが理解できる形式に変換するための2つの関数が定義されています。

## 1. トークン化とは？

テキストデータを数値（トークン）に変換する処理です。これは人間の言葉をAIが理解できる「数字の言葉」に翻訳する作業だと考えてください。

## 2. `tokenize_function`の解説

```python
def tokenize_function(examples):
    result = tokenizer(
        examples['text'],
        truncation=True,
        max_length=TOKENIZE_MAX_LENGTH,
        padding='max_length',
        add_special_tokens=True,
        return_tensors=None,
    )
    return result
```

### パラメータの説明：

1. `truncation=True`
   - 長すぎるテキストを切り詰める
   - 例：長い哲学的な議論を適切な長さに調整

2. `max_length=TOKENIZE_MAX_LENGTH`
   - テキストの最大長を設定
   - 例：「知識とは何か？」という短い質問から、長い対話まで一定の長さに統一

3. `padding='max_length'`
   - 短いテキストを最大長まで埋める
   - 例：
     ```
     短いテキスト：「なぜ？」
     ↓
     パディング後：「なぜ？」 + [PAD][PAD][PAD]...
     ```

4. `add_special_tokens=True`
   - 特別な意味を持つトークンを追加
   - 例：
     ```
     入力：「人生の意味とは何ですか？」
     ↓
     変換後：[START]人生の意味とは何ですか？[END]
     ```

## 3. `preprocess_function`の解説

```python
def preprocess_function(examples):
    """シンプルな前処理"""
    return tokenizer(
        examples['text'],
        truncation=True,
        max_length=TOKENIZE_MAX_LENGTH,
        padding='max_length',
        add_special_tokens=True,
        return_tensors=None
    )
```

この関数は`tokenize_function`と同じ処理を行います。2つの関数が存在する理由は：
- データの異なる処理段階で使用
- 将来的な機能拡張に備える

### 具体例：

ソクラテス式の対話の処理例：
```
元のテキスト：
ユーザー：「真実とは何でしょうか？」
ソクラテスAI：「興味深い質問ですね。あなたにとって真実とは何を意味しますか？」

↓ トークン化処理

[START]
[USER]真実とは何でしょうか？[/USER]
[ASSISTANT]興味深い質問ですね。あなたにとって真実とは何を意味しますか？[/ASSISTANT]
[END]
[PAD][PAD]...（必要に応じてパディング）
```

このように、人間の対話をAIが学習しやすい形式に変換することで、より自然なソクラテス式の対話が可能になります。

        
</div>
    </details>
</div>


In [ ]:
### 2.4 データセットのトークン化と検証
# Optimize dataset processing
tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    batch_size=16,
    num_proc=2,
    load_from_cache_file=True,
    desc="Tokenizing datasets",
    remove_columns=dataset.column_names,
)

# Add dataset validation
def validate_dataset(dataset):
    # Check first element
    first_item = dataset[0]
    print("Validated first item structure:")
    print(f"Keys: {first_item.keys()}")
    print(f"input_ids type: {type(first_item['input_ids'])}")
    print(f"input_ids length: {len(first_item['input_ids'])}")
    return dataset

tokenized_dataset = validate_dataset(tokenized_dataset)


<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">



# データセットのトークン化と検証の説明

このコードセクションは、データセットを実際にAIが学習できる形に変換し、その品質を確認する重要な部分です。

## 1. データセットのトークン化

```python
tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    batch_size=16,
    num_proc=2,
    load_from_cache_file=True,
    desc="Tokenizing datasets",
    remove_columns=dataset.column_names,
)
```

### パラメータの説明：

1. `batched=True, batch_size=16`
   - 16件ずつまとめて処理
   - 例：100個の対話データがあれば、16件ずつ6回（最後は4件）に分けて処理

2. `num_proc=2`
   - 2つのプロセスで並列処理
   - 処理速度を向上させる

3. `load_from_cache_file=True`
   - 一度処理したデータを保存
   - 次回から高速に読み込み可能

4. `remove_columns=dataset.column_names`
   - 元のテキストデータを削除
   - トークン化されたデータのみを保持

## 2. データセットの検証

```python
def validate_dataset(dataset):
    first_item = dataset[0]
    print("Validated first item structure:")
    print(f"Keys: {first_item.keys()}")
    print(f"input_ids type: {type(first_item['input_ids'])}")
    print(f"input_ids length: {len(first_item['input_ids'])}")
    return dataset
```

### 具体例：

元の対話データ：
```
ユーザー：「知恵とは何でしょうか？」
ソクラテスAI：「その質問について、あなたはどのように考えていますか？」
```

トークン化後のデータ構造：
```python
{
    'input_ids': [1, 1543, 235, 8867, ..., 2],  # テキストを数値に変換
    'attention_mask': [1, 1, 1, 1, ..., 0]      # どの部分が実際のテキストか示す
}
```

### 検証内容：

1. データ構造の確認
   ```python
   print(f"Keys: {first_item.keys()}")
   # 期待される出力：Keys: ['input_ids', 'attention_mask']
   ```

2. データ型の確認
   ```python
   print(f"input_ids type: {type(first_item['input_ids'])}")
   # 期待される出力：input_ids type: <class 'list'>
   ```

3. データ長の確認
   ```python
   print(f"input_ids length: {len(first_item['input_ids'])}")
   # 期待される出力：input_ids length: 256（設定した最大長）
   ```

このような検証により：
- データが正しくトークン化されているか
- AIモデルが学習できる形式になっているか
- データの長さが適切か

を確認することができます。これにより、ソクラテス式チャットボットの学習データの品質を保証します。

        
</div>
    </details>
</div>
